## 연구 보드판
- 일단 감정지수가 가격 변동에 유용한가를 봐야한다
즉 가격이아니라 가격 변동에 영향이다
- 가격변동을 기준으로 한다.
- 혹은 z-value 변동을 기준으로한다.
- 만약에 추세의 변화를 알려고 하면 어떻게 해야하나?

- X(t) = 가격
- Y(t) = 심리
- N(t) = 글 수
- I(t) = 인덱스

## I. 패키지 밎 자료 불러오기

In [ ]:
!pip install pymysql
!pip install sqlalchemy
!pip install mysqlclient

     |████████████████████████████████| 43 kB 2.1 MB/s 
     |████████████████████████████████| 87 kB 6.2 MB/s 
  Created wheel for mysqlclient: filename=mysqlclient-2.1.0-cp37-cp37m-linux_x86_64.whl size=99961 sha256=99d3b5423b4a4336e065c2b9e23691b6f8af38f2105c062d688d7ae223b11abf
  Stored in directory: /root/.cache/pip/wheels/97/d4/df/08cd6e1fa4a8691b268ab254bd0fa589827ab5b65638c010b4
Successfully built mysqlclient


In [ ]:
!pip install pyecharts
!pip install chart_studio

     |████████████████████████████████| 135 kB 25.2 MB/s 
     |████████████████████████████████| 130 kB 44.7 MB/s 
     |████████████████████████████████| 64 kB 3.0 MB/s 


In [ ]:
import warnings
warnings.filterwarnings('ignore')
import time
import datetime
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import pymysql
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import font_manager, rc

import itertools
import operator

import base64
from io import BytesIO

from datetime import datetime, timedelta

from pyecharts.charts import Bar
from pyecharts.charts import Line
import plotly.offline as plyo
import cufflinks
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 불러오기
XRP = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/XRP_only.csv')

In [ ]:
XRP.tail()

,Unnamed: 0,date,close_KRW,volume,positive_E,neutrual_E,negative_E,positive_K,neutrual_K,negative_K,positive_ratio_E,positive_ratio_K
1521,1521,2021-11-27,1215.0,1.644314e+08,3142.0,14.0,6397.0,13.0,6.0,20.0,0.328902,0.333333
1522,1522,2021-11-28,1220.0,2.488932e+08,NaN,NaN,NaN,10.0,8.0,17.0,NaN,0.285714
1523,1523,2021-11-29,1230.0,6.381037e+06,NaN,NaN,NaN,8.0,3.0,10.0,NaN,0.380952
1524,1524,2021-11-30,1255.0,7.274336e+06,908.0,1.0,1934.0,4.0,3.0,2.0,0.319381,0.444444
1525,1525,2021-12-01,1250.0,6.126219e+06,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000


## II.전처리


### 1) 영어 제외하기, 날짜 수정 약 371일 데이터

In [ ]:
XRP1 = XRP.drop(columns=['Unnamed: 0','positive_E','neutrual_E','negative_E','positive_ratio_E'])

In [ ]:
xrp = XRP1[-371:-1]

In [ ]:
xrp

,date,close_KRW,volume,positive_K,neutrual_K,negative_K,positive_ratio_K
1155,2020-11-26,600.0,1.706453e+09,45.0,25.0,37.0,0.420561
1156,2020-11-27,627.0,1.029245e+09,26.0,13.0,18.0,0.456140
1157,2020-11-28,688.0,8.124706e+08,13.0,6.0,10.0,0.448276
1158,2020-11-29,665.0,4.950927e+08,16.0,8.0,8.0,0.500000
1159,2020-11-30,720.0,6.329513e+08,20.0,15.0,19.0,0.370370
...,...,...,...,...,...,...,...
1520,2021-11-26,1295.0,1.024812e+07,6.0,1.0,14.0,0.285714
1521,2021-11-27,1215.0,1.644314e+08,13.0,6.0,20.0,0.333333
1522,2021-11-28,1220.0,2.488932e+08,10.0,8.0,17.0,0.285714
1523,2021-11-29,1230.0,6.381037e+06,8.0,3.0,10.0,0.380952


## III.  가격 D1,D2추가 , P/N P/(P+n+N)

In [ ]:
xrp1 = xrp.loc[::-1]

In [ ]:
xrp = xrp1.reset_index(drop=True)

In [ ]:
xrp

,date,close_KRW,volume,positive_K,neutrual_K,negative_K,positive_ratio_K
0,2021-11-30,1255.0,7.274336e+06,4.0,3.0,2.0,0.444444
1,2021-11-29,1230.0,6.381037e+06,8.0,3.0,10.0,0.380952
2,2021-11-28,1220.0,2.488932e+08,10.0,8.0,17.0,0.285714
3,2021-11-27,1215.0,1.644314e+08,13.0,6.0,20.0,0.333333
4,2021-11-26,1295.0,1.024812e+07,6.0,1.0,14.0,0.285714
...,...,...,...,...,...,...,...
365,2020-11-30,720.0,6.329513e+08,20.0,15.0,19.0,0.370370
366,2020-11-29,665.0,4.950927e+08,16.0,8.0,8.0,0.500000
367,2020-11-28,688.0,8.124706e+08,13.0,6.0,10.0,0.448276
368,2020-11-27,627.0,1.029245e+09,26.0,13.0,18.0,0.456140


In [ ]:
xrp['P/N'] = (xrp['positive_K'])/(xrp['negative_K'])
xrp['D(1)P/N(t)'] = -1*(xrp['P/N'].diff().shift(-1))/1
xrp['D(2)P/N(t)'] = -1*(xrp['D(1)P/N(t)'].diff().shift(-1))/1

In [ ]:
xrp['D(1)X(t)'] = -1*(xrp['close_KRW'].diff().shift(-1))/1
xrp['D(2)X(t)'] = -1*(xrp['D(1)X(t)'].diff().shift(-1))/1

In [ ]:
xrp['P/(P+N)'] = (xrp['positive_K'])/((xrp['negative_K'])+(xrp['positive_K']))
xrp['D(1)P/(P+N)(t)'] = -1*(xrp['P/(P+N)'].diff().shift(-1))/1
xrp['D(2)P/(P+N)(t)'] = -1*(xrp['D(1)P/(P+N)(t)'].diff().shift(-1))/1

감정지수

In [ ]:
a=0.9
b=0.09
c=0.01
xrp['I_(P/N)(t)'] = a*(xrp['P/(P+N)'])+b*(xrp['D(1)P/(P+N)(t)'])+c*(xrp['D(2)P/(P+N)(t)'])

In [ ]:
pd.options.display.max_rows = 40
pd.options.display.max_columns = 40

### 1) n 다음날 추론하기

In [ ]:
xrp_n = xrp

In [ ]:
xrp_n['close_KRW1'] = (xrp_n['close_KRW'].shift(1))
xrp_n['D(1)X(t)1'] = (xrp_n['D(1)X(t)'].shift(1))
xrp_n['D(2)X(t)1'] = (xrp_n['D(2)X(t)'].shift(1))

In [ ]:
xrp_n

,date,close_KRW,volume,positive_K,neutrual_K,negative_K,positive_ratio_K,P/N,D(1)P/N(t),D(2)P/N(t),D(1)X(t),D(2)X(t),P/(P+N),D(1)P/(P+N)(t),D(2)P/(P+N)(t),close_KRW1,D(1)X(t)1,D(2)X(t)1
0,2021-11-30,1255.0,7.274336e+06,4.0,3.0,2.0,0.444444,2.000000,1.200000,0.988235,25.0,15.0,0.666667,0.222222,0.148148,NaN,NaN,NaN
1,2021-11-29,1230.0,6.381037e+06,8.0,3.0,10.0,0.380952,0.800000,0.211765,0.273529,10.0,5.0,0.444444,0.074074,0.097643,1255.0,25.0,15.0
2,2021-11-28,1220.0,2.488932e+08,10.0,8.0,17.0,0.285714,0.588235,-0.061765,-0.283193,5.0,85.0,0.370370,-0.023569,-0.117508,1230.0,10.0,5.0
3,2021-11-27,1215.0,1.644314e+08,13.0,6.0,20.0,0.333333,0.650000,0.221429,1.792857,-80.0,-85.0,0.393939,0.093939,0.460606,1220.0,5.0,85.0
4,2021-11-26,1295.0,1.024812e+07,6.0,1.0,14.0,0.285714,0.428571,-1.571429,-1.904762,5.0,10.0,0.300000,-0.366667,-0.408333,1215.0,-80.0,-85.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,2020-11-30,720.0,6.329513e+08,20.0,15.0,19.0,0.370370,1.052632,-0.947368,-1.647368,55.0,78.0,0.512821,-0.153846,-0.255295,681.0,-39.0,-94.0
366,2020-11-29,665.0,4.950927e+08,16.0,8.0,8.0,0.500000,2.000000,0.700000,0.844444,-23.0,-84.0,0.666667,0.101449,0.127141,720.0,55.0,78.0
367,2020-11-28,688.0,8.124706e+08,13.0,6.0,10.0,0.448276,1.300000,-0.144444,-0.372673,61.0,34.0,0.565217,-0.025692,-0.067820,665.0,-23.0,-84.0
368,2020-11-27,627.0,1.029245e+09,26.0,13.0,18.0,0.456140,1.444444,0.228228,NaN,27.0,NaN,0.590909,0.042129,NaN,688.0,61.0,34.0


In [ ]:
corr = xrp.corr(method='pearson')
print(corr)

                  close_KRW    volume  positive_K  neutrual_K  negative_K  \
close_KRW          1.000000 -0.063549   -0.011355   -0.082420   -0.147206   
volume            -0.063549  1.000000    0.770656    0.698618    0.742539   
positive_K        -0.011355  0.770656    1.000000    0.867660    0.901165   
neutrual_K        -0.082420  0.698618    0.867660    1.000000    0.896818   
negative_K        -0.147206  0.742539    0.901165    0.896818    1.000000   
positive_ratio_K   0.153014 -0.054527    0.074234   -0.246012   -0.190156   
P/N                0.127064 -0.151534   -0.118053   -0.226031   -0.301523   
D(1)P/N(t)        -0.006746 -0.022921    0.027085   -0.020952   -0.092030   
D(2)P/N(t)        -0.002299 -0.011272    0.026291    0.005927   -0.036250   
D(1)X(t)           0.086743  0.016324   -0.021024   -0.079727   -0.091353   
D(2)X(t)           0.000975 -0.025850   -0.062562   -0.020929    0.002853   
P/(P+N)            0.187679 -0.101706   -0.017865   -0.187026   -0.294807   

## IV. z-value 연구

In [ ]:
n = [7,14,30,60,90,180]
crypto = [xrp]

for j in crypto :
  for i in n:
    j['MA{0}'.format(i)]= j['close_KRW'].rolling(window=i).mean().shift(-(i-1))

In [ ]:
n = [7,14,30,60,90,180]
crypto = [xrp]
for j in crypto:
  for i in n :
    j['SD{0}(t)'.format(i)]= j['close_KRW'].rolling(window=i).std().shift(-(i-1))
  for i in n :
    j['MA{0}z(t)'.format(i)] = (j['close_KRW']-j['MA{0}'.format(i)])/(j['SD{0}(t)'.format(i)])

In [ ]:
corr = xrp.corr(method='pearson')
print(corr)

### 2) 일단 z-value 매력도 모이는 날 측정 - z밸류 정규화

*   z 밸류 상단 하단이 MA마다 다르다 그래서 정규화를 한번더 한다




In [ ]:
n = [7,14,30,60,90,180]
crypto = [xrp]
for j in crypto:
  for i in n :
    j['MA{0}zMM(t)'.format(i)]=(j['MA{0}z(t)'.format(i)]-j['MA{0}z(t)'.format(i)].min())/(j['MA{0}z(t)'.format(i)].max()-j['MA{0}z(t)'.format(i)].min())

In [ ]:
corr = xrp.corr(method='pearson')
print(corr['MA7zMM(t)'])

close_KRW      0.104057
volume         0.128495
positive_K     0.070641
neutrual_K    -0.057218
negative_K    -0.064894
                 ...   
MA14zMM(t)     0.850774
MA30zMM(t)     0.626416
MA60zMM(t)     0.490842
MA90zMM(t)     0.404919
MA180zMM(t)    0.176054
Name: MA7zMM(t), Length: 42, dtype: float64


### 4) I 로만 측정하기

In [ ]:
# 감정지수만
crypto = [xrp]
N = 0
D = 0
Xt = 0

MMa = 0.2
Ia = 0.65
# for j in crypto:
for i in range(len(xrp)) :
    if (xrp['I_(P/N)(t)'][i] >= Ia) : 


      print(xrp['date'][i])
      print('감정지수', xrp['I_(P/N)(t)'][i])
      a = xrp['close_KRW'][i-1]-xrp['close_KRW'][i]
      print('다음날 결과:', a, '\n')
      N += 1
      Xt += a
      if a >= 0:
        D += 1

print('감정지수 승률 :' ,D/N)
print('누적 :' , Xt)

2021-11-23
감정지수 0.6524864864864864
다음날 결과: -35.0 

2021-11-19
감정지수 0.6813210702341136
다음날 결과: 10.0 

2021-11-15
감정지수 0.7844047619047619
다음날 결과: -85.0 

2021-11-08
감정지수 0.6863636363636363
다음날 결과: -30.0 

2021-11-05
감정지수 0.7544201000238152
다음날 결과: -10.0 

2021-11-02
감정지수 0.6767251461988305
다음날 결과: 100.0 

2021-11-01
감정지수 0.6565497076023391
다음날 결과: 35.0 

2021-10-31
감정지수 0.7273830409356726
다음날 결과: -20.0 

2021-10-29
감정지수 0.6891725105189341
다음날 결과: -15.0 

2021-10-17
감정지수 0.661390977443609
다음날 결과: -10.0 

2021-10-09
감정지수 0.7100266796022313
다음날 결과: -10.0 

2021-10-08
감정지수 0.7182594824530308
다음날 결과: 120.0 

2021-10-06
감정지수 0.6523076923076923
다음날 결과: -10.0 

2021-10-04
감정지수 0.6532857142857144
다음날 결과: 30.0 

2021-09-29
감정지수 0.7248253968253969
다음날 결과: 25.0 

2021-09-23
감정지수 0.6694444444444445
다음날 결과: -65.0 

2021-09-22
감정지수 0.7274444444444446
다음날 결과: -25.0 

2021-09-14
감정지수 0.7673809523809524
다음날 결과: 15.0 

2021-08-28
감정지수 0.7773992673992675
다음날 결과: -10.0 

2021-08-26
감정지수 0.6629326923076924
다음